In [27]:
from gymnasium_sync import TrainingLoop
training_loop = TrainingLoop(num_envs=1, num_steps=50)
training_loop.run()

# Bandit Test

In [28]:
from bandit import Bandit

bandit = Bandit(mode='argmax', l=0, r=1, acc=10, acc2=2, width=2, lr=0.1, d=2)
# bandit.update(x=0.5, g=1)
candidates = bandit.sample()

print("Candidates:", candidates)

Candidates: [0.8 1. ]


In [158]:
from config import config_params

print(config_params["Acrobot-v1"])

{'num_frames': 200000000, 'replay': 2, 'num_envs': 160, 'seq_length': 80, 'bootstrap': 5, 'batch_size': 64, 'discount': 0.997, 'v_loss_scaling': 1.0, 'q_loss_scaling': 10.0, 'pi_loss_scaling': 10.0, 'importance_sampling_clip_c': 1.05, 'importance_sampling_clip_rho': 1.05, 'backbone': 'IMPALA,deep', 'lstm_units': 256, 'optimizer': 'Adam Weight Decay', 'weight_decay_rate': 0.01, 'learning_rate': 0.0005, 'warmup_steps': 4000, 'adamw_beta1': 0.9, 'adamw_beta2': 0.98, 'adamw_epsilon': 1e-06, 'adamw_clip_norm': 50.0, 'learner_push_model_every_n_steps': 25, 'actor_pull_model_every_n_steps': 64, 'bandit_params': {'mode': ['argmax', 'random'], 'tau1': [0.0, 50.0], 'tau2': [0.0, 50.0], 'epsilon': [0.0, 1.0], 'acc': [50, 50, 10], 'acc2': [2, 3, 4], 'width': [2, 3, 4], 'lr': [0.05, 0.1, 0.2], 'd': 3}}


In [195]:
from config import config_params
from bandit import initialize_bandits
from bandit import update_bandits
from bandit import get_candidates
from bandit import sample_candidate

env_name = "Acrobot-v1"
params = config_params[env_name]

bandits = initialize_bandits(params["bandit_params"])

In [231]:
update_bandits(bandits, 22.666666666666664, 9.666666666666666, 0.8500000000000001, 0.5)

In [199]:
bandits[27]['tau1'].sample()

array([10.5, 12.5, 29. ])

In [234]:
all_candidates = get_candidates(bandits)
sampled_candidate = sample_candidate(all_candidates)
print(sampled_candidate)

(22.0, 9.25, 0.25)


# Actor Test

In [114]:
import torch
from actor import Actor
from config import config_params

env_name = "Acrobot-v1"
params = config_params[env_name]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

actor = Actor(params['architecture_params'], device)


In [115]:
obs = [[1,2,3,4,5,6], [2,3,4,5,6,7]]
ind = [[1,1,.4], [1,1,.6]]

In [138]:
v1, v2, a1, a2, policy = actor.calculate_values(obs, ind)
v1, v2, a1, a2, policy

(tensor([[0.0998],
         [0.1613]], device='cuda:0', grad_fn=<AddmmBackward0>),
 tensor([[0.3956],
         [0.4774]], device='cuda:0', grad_fn=<AddmmBackward0>),
 tensor([[-0.4400,  0.5586,  0.2163],
         [-0.4932,  0.6671,  0.2509]], device='cuda:0',
        grad_fn=<AddmmBackward0>),
 tensor([[0.3553, 0.1147, 0.0667],
         [0.5267, 0.2046, 0.0848]], device='cuda:0', grad_fn=<AddmmBackward0>),
 tensor([[0.3075, 0.3785, 0.3140],
         [0.2643, 0.4265, 0.3092]], device='cuda:0', grad_fn=<AddBackward0>))

In [132]:
action, action_prob = actor.get_action(policy, stochastic=True)
action, action_prob

(tensor([2, 1], device='cuda:0'),
 tensor([[0.3140],
         [0.4265]], device='cuda:0', grad_fn=<GatherBackward0>))